What are descriptors?

The underpinning mechanism for properties, slots, and even functions.

__The Problem__

In [1]:
# Having property functions and setters for eah attribute gets tedious and repetitive
class Point:
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self._x = int(value)
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self._y = int(value)
        
    def __init__(self, x, y):
        self.x = x
        self.y = y

__The Descriptor Protocol__

There are 4 main methods to implement the descriptor protocol (they are not all required)
- `__get__`
- `__set__`
- `__delete__`
- `__set_name__`

There are also 2 categories of descriptors:
- non-data descriptors: only implement `__get__`
- data descriptors: implement `__set__` and/or `__delete__`

__Using a Descriptor Class__

In [2]:
from datetime import datetime

In [3]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

In [4]:
class Logger:
    current_time = TimeUTC()

In [5]:
l = Logger()
l.current_time

'2022-02-05T16:54:48.035985'

**The `__get__` Method**

Logger defines a single instance of TimeUTC as a *class attribute*. But because TimeUTC implements `__get__`, Python will use that method when retrieving the instance attribute value.

We can access current_time from the class or the instance:
- if called from an instance, the 'instance' parameter will not be `None`
- the 'owner_class' parameter describes the class that owns the TimeUTC instance

We can return different values from `__get__` depending if it was called from the class or the instance.

If called from the class, return the descriptor instance.

If called from an instance, return the attribute value.

In [6]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if not instance:
            return self
        else:
            return datetime.utcnow().isoformat()

**The `__set__` Method**

The `__set__` method signature differs slightly from `__get__`:

```
def __set__(self, instance, value): ...
```

Since setters are always called from instances, there is no need for the 'owner_class' parameter.

__Caveat with Set/Delete/Get__

When creating multiple instances of a class with descriptors, since the descriptors are defined as class attributes, they will all use the same reference. This is mostly a problem for Set and Delete.

This is one of the reasons why we pass the 'instance' parameter to these dunder methods, so we can be aware of which instance we are storing values for.